In [31]:
import tensorflow
from tensorflow.keras.layers import Reshape
from keras.layers import Input, Embedding, Reshape, LSTM, Dense, Flatten
from keras.models import Model
import os
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

!ls

drive  sample_data


In [32]:
batch_size = 16
max_sequence_length = 384
max_word_length = 20
embedding_size = 128
lstm_units = 256
num_classes = 14
directory_train='/content/drive/MyDrive/Ders/training-data'
directory_test='/content/drive/MyDrive/Ders/testing-data'
directory_all='/content/drive/MyDrive/Ders/all-data'
#'/Users/talhaunal/Downloads/OCR_dataset/training-data'
#'/Users/talhaunal/Downloads/OCR_dataset/testing-data'

In [33]:
train_test_texts=[]

for filename in os.listdir(directory_all):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory_all, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            content = re.sub(r'<start_".*?">', '', content)
            content = re.sub(r'<end_".*?">', '', content)
            txt_data = content.split()
            train_test_texts.extend(txt_data)

tokenizer = Tokenizer(char_level=True, lower=False)
tokenizer.fit_on_texts(train_test_texts)


In [34]:
def process_text_files(directory):
    all_texts = []
    all_texts_extend = []

    # Iterate over all files in the directory


    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                content = re.sub(r'<start_".*?">', '', content)
                content = re.sub(r'<end_".*?">', '', content)
                txt_data = content.split()
                all_texts.append(txt_data)
                all_texts_extend.extend(txt_data)

    #tokenizer = Tokenizer(char_level=True, lower=False)
    #tokenizer.fit_on_texts(all_texts_extend)

    all_sequences=[]
    for i in range(len(all_texts)):
        all_sequences.append(tokenizer.texts_to_sequences(all_texts[i]))

    max_word_length = 20
    trunc_type = 'post'
    padding_type = 'post'

    # Pad each word sequence to the length of the longest word
    all_padded_sequences=[]
    for i in range(len(all_sequences)):
        padded_sequence = pad_sequences(all_sequences[i], maxlen=max_word_length, padding=padding_type, truncating=trunc_type)
        all_padded_sequences.append(padded_sequence)

    return all_padded_sequences, tokenizer.word_index




#   TRAIN
directory_path = directory_train
padded_sequences, word_index = process_text_files(directory_path)
padded_arrays = []
for arr in padded_sequences:
    pad_width = ((0, max_sequence_length - len(arr)), (0, 0))
    padded_arr = np.pad(arr, pad_width, mode='constant', constant_values=0)
    padded_arrays.append(padded_arr)

#  TEST
directory_path_test = directory_test
padded_sequences_test, word_index_test = process_text_files(directory_path_test)
padded_arrays_test = []
for arr in padded_sequences_test:
    pad_width = ((0, max_sequence_length - len(arr)), (0, 0))
    padded_arr = np.pad(arr, pad_width, mode='constant', constant_values=0)
    padded_arrays_test.append(padded_arr)


np.set_printoptions(threshold=np.inf)

#son array'i numpy array'ine çevir
x_train=np.array(padded_arrays)
x_test=np.array(padded_arrays_test)

print(x_train.shape)
print(x_test.shape)

np.set_printoptions(threshold=1000)

(547, 384, 20)
(547, 384, 20)


In [35]:


global wordsList,wordsIndextList

wordsList = []
wordsIndextList = []
max_sequence_length = 384


label_list = ["<start_\"company_name\">",
              "<end_\"company_name\">",
                  "<start_\"date\">",
                  "<end_\"date\">",
                  "<start_\"time\">",
                  "<end_\"time\">",
                  "<start_\"receipt_number\">",
                  "<end_\"receipt_number\">",
                  "<start_\"tax\">",
                  "<end_\"tax\">",
                  "<start_\"amount\">",
                  "<end_\"amount\">"]

label_list2= ["Pad","Others","B_Comp","l_Comp","B_Date","l_Date","B_Time","l_Time",
                  "B_Receipt","l_Receipt","B_Tax","l_Tax","B_Amount","l_Amount"]



def one_hot_encode_index_list():
    oneHotEncodedList = []
    unique_labels_len = len(sorted(set(label_list2)))
    for i in wordsIndextList:
        bitlist = list(0 for i in range(unique_labels_len))
        bitlist[i] = 1
        oneHotEncodedList.append(bitlist)

    for i in  range(max_sequence_length-len(oneHotEncodedList)):
        oneHotEncodedList.append([1,0,0,0,0,0,0,0,0,0,0,0,0,0])
    return oneHotEncodedList


def splitfunction(text:str):
    global wordsList,wordsIndextList
    wordsList = []
    wordsIndextList = []
    words = text.split()
    for word in words:
        wordsList.append(word)
        wordsIndextList.append(1)



def indexAssignment(tag,last):
    if(tag==None):
        return None
    index = label_list.index(tag)+2
    if(last == "l"):
       index +=1
    return index


def checkTag(word:str):
    for tag in label_list:
        index = word.find(tag)
        if index != -1:
            return tag
    return None



def tagingWords():
    control = None
    counter = 0
    for word in wordsList:
        tag = checkTag(word)
        if tag != None:
            if(word.find("<start_") != -1):
                control = tag
            if(word.find("<start_") != -1 and word.find("<end_") == -1):
                wordsIndextList[counter] = indexAssignment(tag,"B")
                counter+=1
            elif(word.find("<start_") != -1 and word.find("<end_") != -1):
                wordsIndextList[counter] = indexAssignment(tag,"B")
                counter+=1
            else:
                wordsIndextList[counter] = indexAssignment(tag,"B")
                counter+=1
            if(word.find("<end_") != -1):
                control = None
        elif control !=None:
            wordsIndextList[counter] = indexAssignment(control,"l")
            counter+=1
        else:
            counter+=1


def oneHotEncodedFunction(texts):
    splitfunction(texts)
    tagingWords()
    return np.array(one_hot_encode_index_list())



# TRAIN
folder_path_train =  directory_train
file_list_train = os.listdir(folder_path_train)
y_train= []
for file_name in file_list_train:
    if file_name.endswith(".txt"):
        file_path = os.path.join(folder_path_train, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            result = oneHotEncodedFunction(text)

            if len(y_train) == 0:
                y_train = np.array([result])
            else:
                y_train = np.concatenate((y_train, [result]), axis=0)


# TEST
folder_path_test =  directory_test
file_list_test = os.listdir(folder_path_test)
y_test= []
for file_name in file_list_test:
    if file_name.endswith(".txt"):
        file_path = os.path.join(folder_path_test, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            result = oneHotEncodedFunction(text)

            if len(y_test) == 0:
                y_test = np.array([result])
            else:
                y_test = np.concatenate((y_test, [result]), axis=0)


print(y_train.shape)
print(y_test.shape)

(547, 384, 14)
(547, 384, 14)


In [36]:
# LSTM Modeli

input_shape = (max_sequence_length, max_word_length)
input_layer = Input(shape=(max_sequence_length,max_word_length))
print(input_layer.shape)


(None, 384, 20)


In [37]:
char_embedding = Embedding(input_dim=max_word_length, output_dim=embedding_size)(input_layer)
print(char_embedding.shape)


(None, 384, 20, 128)


In [38]:
reshaped_embedding = tensorflow.reshape(char_embedding, (-1, max_word_length, embedding_size))
print(reshaped_embedding.shape)

(None, 20, 128)


In [39]:
char_lstm, state_h, state_c = LSTM(units=lstm_units, return_sequences=True, return_state=True)(reshaped_embedding)
print(char_lstm)

char_lstm2, state_h, state_c = LSTM(units=lstm_units, return_sequences=True, return_state=True)(char_lstm, initial_state=[state_h, state_c])
print(char_lstm2)

KerasTensor(type_spec=TensorSpec(shape=(None, 20, 256), dtype=tf.float32, name=None), name='lstm_4/PartitionedCall:1', description="created by layer 'lstm_4'")
KerasTensor(type_spec=TensorSpec(shape=(None, 20, 256), dtype=tf.float32, name=None), name='lstm_5/PartitionedCall:1', description="created by layer 'lstm_5'")


In [40]:
reshaped = tensorflow.reshape(state_h, (-1,max_sequence_length, state_h.shape[-1]))
print(reshaped.shape)

(None, 384, 256)


In [41]:
word_lstm1, state_h, state_c = LSTM(units=lstm_units, return_sequences=True, return_state=True)(reshaped)
print(word_lstm1)
word_lstm2, state_h, state_c = LSTM(units=lstm_units, return_sequences=True, return_state=True)(word_lstm1, initial_state=[state_h, state_c])
print(word_lstm2)

KerasTensor(type_spec=TensorSpec(shape=(None, 384, 256), dtype=tf.float32, name=None), name='lstm_6/PartitionedCall:1', description="created by layer 'lstm_6'")
KerasTensor(type_spec=TensorSpec(shape=(None, 384, 256), dtype=tf.float32, name=None), name='lstm_7/PartitionedCall:1', description="created by layer 'lstm_7'")


In [42]:
classification_output = Dense(units=num_classes, activation='softmax')(word_lstm2)
print(word_lstm2)

KerasTensor(type_spec=TensorSpec(shape=(None, 384, 256), dtype=tf.float32, name=None), name='lstm_7/PartitionedCall:1', description="created by layer 'lstm_7'")


In [43]:
model = Model(inputs=input_layer, outputs=classification_output)

In [44]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 384, 20)]            0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 384, 20, 128)         2560      ['input_2[0][0]']             
                                                                                                  
 tf.reshape_2 (TFOpLambda)   (None, 20, 128)              0         ['embedding_1[0][0]']         
                                                                                                  
 lstm_4 (LSTM)               [(None, 20, 256),            394240    ['tf.reshape_2[0][0]']        
                              (None, 256),                                                  

In [45]:
model.fit(x_train, y_train, epochs=10,batch_size=16, validation_data=(x_test, y_test))

Epoch 1/10
35/35 [==============================] - 25s 487ms/step - loss: 0.8753 - accuracy: 0.7057 - val_loss: 0.4377 - val_accuracy: 0.7915
Epoch 2/10
35/35 [==============================] - 11s 327ms/step - loss: 0.1827 - accuracy: 0.9576 - val_loss: 0.1390 - val_accuracy: 0.9672
Epoch 3/10
35/35 [==============================] - 11s 331ms/step - loss: 0.1363 - accuracy: 0.9675 - val_loss: 0.1253 - val_accuracy: 0.9698
Epoch 4/10
35/35 [==============================] - 11s 328ms/step - loss: 0.1291 - accuracy: 0.9686 - val_loss: 0.1395 - val_accuracy: 0.9642
Epoch 5/10
35/35 [==============================] - 12s 333ms/step - loss: 0.1242 - accuracy: 0.9678 - val_loss: 0.1052 - val_accuracy: 0.9697
Epoch 6/10
35/35 [==============================] - 14s 397ms/step - loss: 0.1048 - accuracy: 0.9697 - val_loss: 0.0959 - val_accuracy: 0.9702
Epoch 7/10
35/35 [==============================] - 14s 396ms/step - loss: 0.1004 - accuracy: 0.9700 - val_loss: 0.0937 - val_accuracy: 0.9702

In [50]:
model.save('model.h5')